In [162]:
import pandas as pd
import numpy as np
from collections import Counter,defaultdict
from evaluators import *
df = pd.read_csv('train.txt',header = None)
Y_train= df[14].values
del df[2]
del df[14]
X_train=df.values

df = pd.read_csv('test.txt',header = None)
Y_test= df[14].values
del df[2]
del df[14]
X_test=df.values


In [163]:
class NaiveBaseClass:
    def get_pro(self,item):
        no_item = len(item)
        dict_item = dict(Counter(item))
        for key in dict_item.keys():
            dict_item[key] = float(dict_item[key])/no_item
        return dict_item
    
    def get_max_value_key(self,d1):
        values = d1.values()
        keys =d1.keys()
        max_value_index = values.index(max(values))
        max_key = keys[max_value_index]
        return max_key
    def initialize_nb_dict(self):
        self.nb_dict ={}
        for label in self.labels:
            self.nb_dict[label] = defaultdict(list)
        
     
class NaiveBayesClassifier(NaiveBaseClass):
    def train(self,X,Y):
        self.labels = np.unique(Y)
        self.initialize_nb_dict()
        self.dict_pro_perClass = self.get_pro(Y)
        for label in self.labels:
            row_indices = np.where(Y==label)[0]
            X_= X[row_indices,:]
            n_rows,n_cols = np.shape(X_)
            for col in range(n_cols):
                self.nb_dict[label][col] += list(X_[:,col])
            
        for label in self.labels:
            for col in range(n_cols):
                self.nb_dict[label][col] = self.get_pro(self.nb_dict[label][col])    
    
    def pro_per_elem(self,X_elem):
        dict_pro = {}
        for label in self.labels:
            class_pro = self.dict_pro_perClass[label]
            for ii in range(len(X_elem)):
                dict_feature_values=self.nb_dict[label][ii]
                if X_elem[ii] in dict_feature_values.keys():
                    class_pro *= dict_feature_values[X_elem[ii]]
                else:
                    class_pro *=0
            dict_pro[label] = class_pro
        return self.get_max_value_key(dict_pro)
    
    def classify(self,X):
        self.predicted_Y_values = []
        rows, cols = np.shape(X)
        for ii in range(rows):
            X_elem = X[ii,:]
            prediction = self.pro_per_elem(X_elem)
            self.predicted_Y_values.append(prediction)
        return self.predicted_Y_values       

In [164]:
nbc = NaiveBayesClassifier()
nbc.train(X_train,Y_train)
print("trained")
predicted_Y = nbc.classify(X_test)
y_labels = np.unique(Y_test)
for y_label in y_labels:
    f1 = f1_score(predicted_Y,Y_test,y_label)
    print ("F1-score on the test-set for class %s is: %s"% (y_label,f1))

trained
F1-score on the test-set for class  <=50K is: 0.894577807621
F1-score on the test-set for class  >50K is: 0.7123883673
